# Storing Text objects in Postgres

This tutorial demonstrates how to store and query estnltk text objects in a postgres database.

In [2]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, JsonbQuery as Q

First, let's connect to an existing postgres database and create a new table:

In [3]:
storage = PostgresStorage(dbname='estnltk', user='distorti', password='kolobog2511')
table = 'tmp'
storage.create_table(table)

Next, we insert some data:

In [4]:
text1 = Text('ööbik laulab.').analyse('morphology')
key1 = storage.insert(table, text1)
print(key1, text1)

text2 = Text('öökull ei laula.').analyse('morphology')
key2 = storage.insert(table, text2, key=7)
print(key2, text2)

1 Text(text="ööbik laulab.")
7 Text(text="öökull ei laula.")


Select all the data in our table:

In [5]:
for key, text in storage.select(table):
    print(key, text)

1 Text(text="ööbik laulab.")
7 Text(text="öökull ei laula.")


Find row by key:

In [6]:
txt = storage.select_by_key(table, key=7)
print(txt)

Text(text="öökull ei laula.")


Search using layer attributes:

In [7]:
q = Q('morph_analysis', lemma='laulma')
for key, txt in storage.select(table, query=q):
    print(key, txt)

1 Text(text="ööbik laulab.")
7 Text(text="öökull ei laula.")


Search using multiple layer attributes:

In [8]:
q = Q('morph_analysis', lemma='laulma', form='b')
for key, txt in storage.select(table, query=q):
    print(key, txt)

1 Text(text="ööbik laulab.")


Search using "OR" query:

In [9]:
q = Q('morph_analysis', lemma='ööbik') | Q('morph_analysis', lemma='öökull')
for key, txt in storage.select(table, query=q):
    print(key, txt)

1 Text(text="ööbik laulab.")
7 Text(text="öökull ei laula.")


Search using "AND" query:

In [10]:
q = Q('morph_analysis', lemma='ööbik') & Q('morph_analysis', lemma='öökull')
for key, txt in storage.select(table, query=q):
    print(key, txt)

Search using a composite query:

In [11]:
q = (Q('morph_analysis', lemma='ööbik') | Q('morph_analysis', lemma='öökull')) & Q('morph_analysis', lemma='laulma')
for key, txt in storage.select(table, query=q):
    print(key, txt)

1 Text(text="ööbik laulab.")
7 Text(text="öökull ei laula.")


Drop table and close connection:

In [12]:
storage.drop_table(table)
storage.close()